<a href="https://colab.research.google.com/github/zeinabmohamed/HistopathologicCancerDetection/blob/master/fisher_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Dec 29 20:51:00 2019

@author: minamounir
"""
import numpy as np
import os
import imageio
import matplotlib.pylab as plt
import copy 
import pandas as pd
import sys
np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(linewidth=sys.maxsize)

def getVectorFromCsv(csvFile):
    data = pd.read_csv(csvFile)
    intArray = np.fromstring(data["imageVector"][0].strip('[]'), dtype=int, sep=' ')
    i = int(data["label"][0])
    return intArray,i


def getImageNameAndLabel(csvFile):
    data = pd.read_csv(csvFile)
    return data
#print(getIVectorFromCsv("./histopathologic-cancer-detection/DataSet2/3e24dd48c3a419f4d669cac3cf71e777ba100541.csv"))


#for i in range(len(Vectors)):

df= getImageNameAndLabel("./histopathologic-cancer-detection/train_labels.csv")
#print(df)

msk = np.random.rand(len(df)) < 0.8

df_train = df[msk]

df_test = df[~msk]

rangOfTrainData = 5000
rangOfTestData = 500

df_train = df_train[-rangOfTrainData:]
df_test = df_test[-rangOfTestData:]

Vectors=[]
counterIndex=0
## prepare train data 
for index, dataRow in df_train.iterrows(): 
    counterIndex+=1
    imageId = dataRow.loc['id']
       
    v = getVectorFromCsv("./histopathologic-cancer-detection/DataSet2/"+imageId+".csv")
    Vectors.append(v)
    if(counterIndex%1000 == 0):
        print(counterIndex*100/rangOfTrainData)

print(Vectors[0][0]) #vector 

test_Vectors=[]
counterIndex=0  
## Prepare test data 
for index, dataRow in df_test.iterrows(): 
    counterIndex+=1
    imageId = dataRow.loc['id']
       
    v = getVectorFromCsv("./histopathologic-cancer-detection/DataSet2/"+imageId+".csv")
    test_Vectors.append(v)
    if(counterIndex%1000 == 0):
        print(counterIndex*100/rangOfTestData)
        
print(test_Vectors[0][0]) #vector 
print(test_Vectors[0][1]) #vector 


#if(Vectors[i][1] == 0):
#print('>>>>>> Mina <<<<<<<')

#print(Vectors[0][0]) #vector   
#print(Vectors[0][1]) #labels
    

C1=[]
C2=[]

def train(L,C1,C2):

    print("L",L)
    print("len(Vectors)",len(Vectors))
    
    for i in range(len(Vectors)):
        if(Vectors[i][1] == L):
            C1.append(Vectors[i][0])
        else:
            C2.append(Vectors[i][0])
    C1= np.asarray(C1)
    #print("mean C1: ",len(np.mean(C1, axis=0)))
    
    C2= np.asarray(C2)
    print(len(C1))
    print(len(C2))
    
        
def getMean(DataSet): 
    return np.mean(DataSet, axis=0)

def MultiplyMatrix(a1,a2,tr):
    if tr==0:
        M1 = np.matrix(a1).T
        M2 = np.matrix(a2)
    else:
        M1 = np.matrix(a1)
        M2 = np.matrix(a2).T    
    return M1*M2



def getS1(C1,m1):
    # Calculate First Term of SW
    sumS1= np.zeros((3072,3072))
    for i in range(len(C1)):
        val =  np.subtract(C1[i], m1)
        #print("val len: ",len(val))
        sumS1 = sumS1 + np.dot(val.T,val) # MultiplyMatrix(val,val,0) 
    return sumS1


def getS2(C2,m2):
    # Calculate First Term of SW
    sumS2= np.zeros((3072,3072))
    for i in range(len(C2)):
        val =  np.subtract(C2[i], m2)
        #print("val len: ",len(val))
        sumS2 = sumS2 + np.dot(val.T,val) #MultiplyMatrix(val,val,0) 
     
   
    return sumS2 


def getSw(S1,S2):
    return np.add(S1,S2)

def get_W(Sw, mean1, mean2):
    m= 10^-6
    return np.dot(np.linalg.inv(Sw+np.eye(Sw.shape[1])*m), (mean2-mean1))
    


def get_Wo(w, m1, m2):
    negative_WT= np.negative(w)
    print("negative_WT : " , len(negative_WT))
    add_means = np.add(m1,m2)
    print("add_means : " , len(add_means))
    return np.dot(negative_WT.T,np.divide(add_means, 2)) #MultiplyMatrix(negative_WT, np.true_divide(add_means, 2),1)
    

W=np.zeros((2,3072))
W_o=np.zeros((2,1))


for Label in range (2):
    print("########## Loop",Label,"#############")
    C1=[]
    C2=[]
    train(Label,C1,C2)

    # get mean of class 1
    m1 = getMean(C1)

    # get mean of class 2
    m2 = getMean(C2)

    print(len(m1))
    print(len(m2))

    S1 = getS1(C1,m1)
    S2 = getS2(C2,m2)
    #print(S1(C1,m1))
    #print("=========================================")
    #print(S2(C2,m2))
    Sw = getSw(S1,S2)
    w = get_W(Sw, m1, m2)
    W[Label]= copy.deepcopy(w)
    #print("SW : ",Sw)

    #print("Weight : ",w)
    w_o = get_Wo(w, m1, m2)
    W_o[Label]= copy.deepcopy(w_o)
    print("Wo : ",w_o)
            

#-----------------Testing---------------
#os.chdir("/Users/minamounir")
    
    
def readAllImagesFromTest_cropped(path):
    files = os.listdir(path)
    for name in files:
     df.append(name)
    return df 


'''
test_Vectors=[]

#for i in range(len(Vectors)):

df= readAllImagesFromTest_cropped("./histopathologic-cancer-detection/test_cropped")
#print(df)

n=100000
for i in range (n): #range(len(df)):
    id = df.loc[i,0]
       
    v = getVectorFromCsv("./histopathologic-cancer-detection/test_cropped/"+id+".csv")
    test_Vectors.append(v)
    if(i%1000 == 0):
        print(i*100/n)
    
print(test_Vectors[0][1]) #vector 
'''

Y_all = np.zeros((len(df_test),2))
Y_weights = np.zeros((2,1))


for i in range(len(test_Vectors)):
    for j in range(2):
        w_traspose = W[j].T #1*3072
        test_row = test_Vectors[i][0] #3072
        Y=np.dot(w_traspose,test_row)+W_o[j]
        Y_weights[j]=Y
    Y_all[i,0]=test_Vectors[i][1]
    Y_all[i,1]=  np.argmax(Y_weights)
    
print(Y_all)
        
#Calculate Accuracy and Print it
accuracy = 0

for i in range(len(Y_all)):
    if (Y_all[i,0] == Y_all[i,1]):
        accuracy = accuracy + 1
 
print("Accuracy = ",((accuracy/rangOfTestData)*100)," %")
    
